#Interactive Visual Analytics with Folium lab

In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=67abef36533722ae301bc0387c96c2c24669e9c96170ab7838d25f7b7e515568
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import folium
import wget
import pandas as pd

In [8]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

##Task 1: Mark all launch sites on a map

In [3]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [4]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(nasa_coordinate,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',))
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
 #For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
locations = launch_sites_df[['Launch Site','Lat','Long']].values.tolist()
for each in locations:
    circle = folium.Circle(each[1:], radius=1000, color='#000000', fill=True)
    marker = folium.map.Marker(
    each[1:],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % each[0],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
display(site_map)

##Task 2: Mark the success/failed launches for each site on the map

In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [12]:
marker_cluster = MarkerCluster()

In [13]:
marker_color = spacex_df[['class']].replace({1: 'green', 0: 'red'})
marker_color

,class
0,red
1,red
2,red
3,red
4,red
5,red
6,red
7,red
8,red
9,red


In [14]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [18]:
site_map.add_child(marker_cluster)
for index, record in spacex_df.iterrows():
    marker = folium.map.Marker(
    record[['Lat','Long']].tolist(),
    ).add_child(folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

##TASK 3: Calculate the distances between a launch site to its proximities

In [19]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [21]:
distance_coastline = calculate_distance(28.563197, -80.576820, 28.56311, -80.56801)

In [22]:
distance_marker = folium.Marker(
   [28.56311, -80.56801],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,20),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
               )
    )

site_map.add_child(distance_marker)

In [23]:
lines=folium.PolyLine(locations=[[28.563197, -80.576820], [28.56311, -80.56801]], weight=1)
site_map.add_child(lines)

In [24]:
distance_city = calculate_distance(28.563197, -80.576820, 28.5411, -81.38178)
distance_city_marker = folium.Marker(
   [28.5411, -81.38178],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,20),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
               )
    )

site_map.add_child(distance_city_marker)

In [25]:
distance_railway = calculate_distance(28.563197, -80.576820, 28.57226, -80.58525)
distance_railway_marker = folium.Marker(
   [28.57226, -80.58525],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,20),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
               )
    )

site_map.add_child(distance_railway_marker)

In [26]:
distance_highway = calculate_distance(28.563197, -80.576820, 28.56315, -80.57078)
distance_highway_marker = folium.Marker(
   [28.56315, -80.57078],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,20),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
               )
    )

site_map.add_child(distance_highway_marker)